<a href="https://colab.research.google.com/github/Swathi1309/ED18B034_ME18B133_Assignment1/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
!wandb login

In [113]:
import keras
from keras.datasets import fashion_mnist
from PIL import Image
import wandb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from tensorflow.keras import initializers
import tensorflow as tf

wandb.init(project="CS6910-assg1", entity="swathi")
classes = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']

In [114]:
def load_data():
  (x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
  X_train = np.transpose((np.array(x_train[:54000],dtype=int).reshape([54000,784]))/255)
  Y_train = np.transpose((np.array(y_train[:54000],dtype=int).reshape([54000,1])))
  X_validation = np.transpose((np.array(x_train[54000:],dtype=int).reshape([6000,784]))/255)
  Y_validation = np.transpose(np.array(y_train[54000:],dtype=int).reshape([6000,1]))
  X_test = np.transpose((np.array(x_test,dtype=int).reshape([10000,784]))/255)
  Y_test = np.transpose(np.array(y_test,dtype=int).reshape([10000,1]))
  return X_train, Y_train, X_validation, Y_validation, X_test, Y_test

In [115]:
def log_images():
  images = []
  labels = []
  for i in range(len(X_train)):
    if len(labels)==len(classes):
      break
    j = random.randint(0,53999)
    if classes[Y_train[0,j]] not in labels:
      images.append(X_train[:,j].reshape([28,28]))
      labels.append(classes[Y_train[0,j]])
  wandb.log({"Examples": [wandb.Image(img, caption=caption) for img,caption in zip(images,labels)]})

Loading data

In [116]:
X_train, Y_train, X_validation, Y_validation, X_test, Y_test = load_data()

Defining basic parameters

In [117]:
N= [0,64,32,16,10]  #Insert the number of neurons in each layer
N[0]=X_train.shape[0]
l = len(N) - 1

Weights initialising

In [119]:
def initialize_weights_random(N, weight_factor):
  np.random.seed(0)
  W = []
  b = []
  W.append(0)
  b.append(0)
  for i in range(1,l+1):
    W.append((np.random.randn(N[i],N[i-1]))*weight_factor)
    b.append(np.zeros((N[i],1)))

  return W,b 

In [120]:
def initialize_weights_xavier(N, weight_factor):
  initializer = tf.keras.initializers.GlorotNormal()
  W = []
  b = []
  W.append(0)
  b.append(0)
  for i in range(1,l+1):
    values = initializer(shape=(N[i],N[i-1]))
    W.append(np.array(values)*weight_factor)
    b.append(np.zeros((N[i],1)))
    del values
  return W,b 

In [162]:
def initialize_weights(N,init, act):
  if act =='relu':
    weight_factor = 10**-2
  else:
    weight_factor = 1
  if init=='random':
    return initialize_weights_random(N, weight_factor)

  if init=='xavier':
    return initialize_weights_xavier(N, weight_factor)

In [122]:
def one_hot(X,y):
  one_hot_y=np.zeros((10,X.shape[1]))
  for i in range(X.shape[1]):
    one_hot_y[y[0,i],i] = 1
  
  return one_hot_y

Softmax

In [123]:
def softmax(a):
  return np.exp(a)/sum(np.exp(a))

Sigmoid

In [124]:
def sigmoid(a):
  return 1/(1+np.exp(-a))

Sigmoid derivative

In [125]:
def dSigmoid(a):
  return sigmoid(a)*(1-sigmoid(a))

Tanh activation function

In [126]:
def tanh(a):
  return np.tanh(a)

Derivative of the tanh function

In [127]:
def dTanh(a):
  return 1- np.square(tanh(a))

ReLU

In [128]:
def ReLU(a):
  return np.maximum(0,a)

Derivative of ReLU

In [129]:
def dReLU(a):
  a[a>=0]=1
  a[a<0]=0
  return a

In [130]:
def activation(a,act):
  if act=='sigmoid':
    return sigmoid(a)
  if act=='tanh':
    return tanh(a)
  if act=='relu':
    return ReLU(a)

def dActivation(a,act):
  if act=='sigmoid':
    return dSigmoid(a)
  if act=='tanh':
    return dTanh(a)
  if act=='relu':
    return dReLU(a)

Cross Entropy

In [131]:
def cross_entropy_loss(y_hat,Y, W, alpha):
  loss = 0
  for i in range(Y.shape[1]):
    loss -= np.log(y_hat[Y[0,i],i])
  norm = 0
  for i in range(1, len(W)):
    norm +=np.linalg.norm((W[i]))
  loss = loss +norm
  loss = loss/Y.shape[1]
  return loss

In [132]:
def d_cross_entropy_loss(y_hat, X, y):
  return (y_hat - one_hot(X,y))

In [133]:
def l2_loss(y_hat,Y,W,alpha):
  loss=(np.linalg.norm(y_hat-Y))
  norm = 0
  for i in range(1, len(W)):
    norm +=np.linalg.norm((W[i]))
  loss = loss +norm
  loss = loss/Y.shape[1]
  return loss

In [134]:
def d_l2_loss(y_hat,X, Y):
  return np.multiply((y_hat - one_hot(X,y)),np.multiply(y_hat,(1-y_hat)))

In [135]:
def Loss(y_hat, y, W, alpha, choose_loss='crossentropy'):
  if choose_loss == 'crossentropy':
    return cross_entropy_loss(y_hat,y,W, alpha)
  if choose_loss =='l2':
    return l2_loss(y_hat,y,W, alpha)

def dLoss(y_hat,X, y, choose_loss='crossentropy'):
  if choose_loss == 'crossentropy':
    return d_cross_entropy_loss(y_hat, X, y)
  if choose_loss =='l2':
    return d_l2_loss(y_hat, X, y)

In [136]:
def accuracy(y_hat,Y):
  return ((y_hat == Y).sum()/Y.shape[1])

Feed forward network

In [137]:
def feedforward(X_input, W, b, N, l, act):
  a=[]
  a.append(np.array([0]))
  h=[]
  h.append(X_input)
  for i in range(1,l):
    a_new=b[i]+np.matmul(W[i],h[i-1])
    h_new=activation(a_new,act)
    a.append(a_new)
    h.append(h_new)

  a_new=b[l]+np.matmul(W[l],h[l-1])
  y_hat=softmax(a_new)
  a.append(a_new)

  return a,h,y_hat

Backpropogation

In [138]:
def backprop(X, y,y_hat,W,a,h,act,alpha):
  grad_W=[]
  grad_b=[]
  grad_a=[]
  grad_h=[]
  
  da = dLoss(y_hat, X, y)
  grad_a.append(da)

  for i in reversed(range(1,l+1)):
    dw = (np.matmul(da,np.transpose(h[i-1])) + alpha*W[i])/(X.shape[1])
    db = np.sum(da, axis=1, keepdims=True)/(X.shape[1])
    dh_prev = np.matmul(np.transpose(W[i]),da)
    da_prev = dh_prev*dActivation(a[i-1],act)

    grad_W.append(dw)
    grad_b.append(db)
    da=da_prev

  grad_W.append(0)
  grad_b.append(0)
  grad_W.reverse()
  grad_b.reverse() 

  return grad_W,grad_b


In [139]:
def loss_and_accuracy(W, b, X, Y, act, alpha):
  y_hat = feedforward(X, W, b, N, l, act)[2]
  y_predict = np.argmax(y_hat,axis=0).reshape([1,X.shape[1]])
  loss=Loss(y_hat, Y, W, alpha)
  acc=accuracy(y_predict,Y)
  return loss,acc

Gradient Descent

In [190]:
def gradient_descent(epochs, learn_rate, batch_size, act, init, N, alpha):
  W, b = initialize_weights(N,init, act)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      a, h, y_hat = feedforward(X, W, b, N, l,act)
      grad_W, grad_b = backprop(X, Y, y_hat, W, a, h, act, alpha)
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for k in range(1,l+1):
        W_new.append(W[k] - (learn_rate*grad_W[k] - learn_rate*alpha*W[k]))
        b_new.append(b[k] - (learn_rate*grad_b[k]))
      W = W_new
      b = b_new
    loss, acc = loss_and_accuracy(W, b, X_train, Y_train, act,alpha)
    val_loss, val_acc = loss_and_accuracy(W, b, X_validation, Y_validation, act,alpha)
    wandb.log({"loss": loss, "val_loss": val_loss, "accuracy": acc, "val_accuracy": val_acc, "epoch": i})
  return W, b

In [177]:
W, b = gradient_descent(10,0.1,32,'relu','random',N, 0.0005)

2.302942667122509 0.09996296296296296
1.860225113467385 0.3048888888888889
0.5874161476560543 0.7827592592592593
0.4776793874802032 0.8227222222222222
0.4188385338243253 0.8415925925925926
0.37158470000451876 0.8648148148148148
0.38379278171080566 0.8587222222222223
0.3671419185334141 0.8658703703703704
0.3779118523820231 0.8653333333333333
0.3269141174971168 0.882537037037037


Momentum based gradient descent

In [160]:
def mom_grad_descent(epochs, learn_rate, gamma, batch_size, act, init, N, alpha):
  W, b = initialize_weights(N,init, act)
  update_prev_w = []
  update_prev_w.append(0)
  update_prev_b = []
  update_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    update_prev_w.append(np.zeros(W[i].shape))
    update_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      update_w = []
      update_w.append(0)
      update_b = []
      update_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      grad_W, grad_b = backprop(X, Y, y_hat, W, a, h, act, alpha)
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      
      for k in range(1,l+1):
        update_w.append(np.zeros(W[k].shape))
        update_b.append(np.zeros(b[k].shape))
        update_w[k] = gamma*update_prev_w[k] + learn_rate*grad_W[k] + learn_rate*alpha*W[k]
        update_b[k] = gamma*update_prev_b[k] + learn_rate*grad_b[k]
        W_new.append((W[k] - update_w[k]/batch_size))
        b_new.append((b[k] - update_b[k]/batch_size))
        update_prev_w[k]=update_w[k]
        update_prev_b[k]=update_b[k]
      del update_w
      del update_b
      W = W_new
      b = b_new
    loss, acc = loss_and_accuracy(W, b, X_train, Y_train, act,alpha)
    val_loss, val_acc = loss_and_accuracy(W, b, X_validation, Y_validation, act,alpha)
    wandb.log({"loss": loss, "val_loss": val_loss, "accuracy": acc, "val_accuracy": val_acc, "epoch": i})
  return W, b

In [ ]:
W, b = mom_grad_descent(10,0.1,0.9,32,'sigmoid','random', N, 0.0005, 1)

Nesterov accelerated gradient descent

In [159]:
def nesterov_grad_descent(epochs, learn_rate, gamma, batch_size, act, init, N, alpha):
  W, b = initialize_weights(N,init, act)
  update_prev_w = []
  update_prev_w.append(0)
  update_prev_b = []
  update_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    update_prev_w.append(np.zeros(W[i].shape))
    update_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      update_w = []
      update_w.append(0)
      update_b = []
      update_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for k in range(1,l+1):
        update_w.append(np.zeros(W[k].shape))
        update_b.append(np.zeros(b[k].shape))
        update_w[k] = gamma*update_prev_w[k]
        update_b[k] = gamma*update_prev_b[k]
        W_new.append((W[k] - update_w[k]/batch_size))
        b_new.append((b[k] - update_b[k]/batch_size))
      W = W_new
      b = b_new
      grad_W, grad_b = backprop(X, Y, y_hat, W, a, h, act, alpha)
      for k in range(1,l+1):
        update_w[k] = gamma*update_prev_w[k] + learn_rate*grad_W[k] + learn_rate*alpha*W[k]
        update_b[k] = gamma*update_prev_b[k] + learn_rate*grad_b[k]
        W_new[k] = (W[k] - update_w[k]/batch_size)
        b_new[k] = (b[k] - update_b[k]/batch_size)
        update_prev_w[k]=update_w[k]
        update_prev_b[k]=update_b[k]
      del update_w
      del update_b
      W = W_new
      b = b_new
    loss, acc = loss_and_accuracy(W, b, X_train, Y_train, act,alpha)
    val_loss, val_acc = loss_and_accuracy(W, b, X_validation, Y_validation, act,alpha)
    wandb.log({"loss": loss, "val_loss": val_loss, "accuracy": acc, "val_accuracy": val_acc, "epoch": i})
  return W, b

In [ ]:
W, b = nesterov_grad_descent(10,0.1,0.9,32,'sigmoid','random',N, 0.0005, 1)

RMS Prop

In [158]:
def rms_prop(epochs, learn_rate, beta, epsilon, batch_size, act, init, N, alpha):
  W, b = initialize_weights(N,init, act)
  update_prev_w = []
  update_prev_w.append(0)
  update_prev_b = []
  update_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    update_prev_w.append(np.zeros(W[i].shape))
    update_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      update_w = []
      update_w.append(0)
      update_b = []
      update_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      grad_W, grad_b = backprop(X, Y, y_hat, W, a, h, act, alpha)
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for k in range(1,l+1):
        update_w.append(np.zeros(W[k].shape))
        update_b.append(np.zeros(b[k].shape))
        update_w[k] = beta*update_prev_w[k] + (1-beta)*np.square(grad_W[k])
        update_b[k] = beta*update_prev_b[k] + (1-beta)*np.square(grad_b[k])
        W_new.append((W[k] - (learn_rate*np.divide(grad_W[k],(np.sqrt(update_w[k]+epsilon)))) - learn_rate*alpha*W[k]))
        b_new.append((b[k] - (learn_rate*np.divide(grad_b[k],(np.sqrt(update_b[k]+epsilon))))))
        update_prev_w[k]=update_w[k]
        update_prev_b[k]=update_b[k]
      del update_w
      del update_b
      W = W_new
      b = b_new
    loss, acc = loss_and_accuracy(W, b, X_train, Y_train, act,alpha)
    val_loss, val_acc = loss_and_accuracy(W, b, X_validation, Y_validation, act,alpha)
    wandb.log({"loss": loss, "val_loss": val_loss, "accuracy": acc, "val_accuracy": val_acc, "epoch": i})
  return W, b

In [ ]:
W, b = rms_prop(10,0.001,0.9,10**-8,16,'tanh','random',N, 0.005, 1)

In [189]:
def adam(epochs, learn_rate, beta1, beta2, epsilon, batch_size, act, init, N, alpha):
  W, b = initialize_weights(N,init, act)
  m_prev_w = []
  m_prev_w.append(0)
  m_prev_b = []
  m_prev_b.append(0)
  v_prev_w = []
  v_prev_w.append(0)
  v_prev_b = []
  v_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    m_prev_w.append(np.zeros(W[i].shape))
    m_prev_b.append(np.zeros(b[i].shape))
    v_prev_w.append(np.zeros(W[i].shape))
    v_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      m_w = []
      m_w.append(0)
      v_w = []
      v_w.append(0)
      m_b = []
      m_b.append(0)
      v_b = []
      v_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      grad_W, grad_b = backprop(X, Y, y_hat, W, a, h, act, alpha)
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for k in range(1,l+1):
        m_w.append(np.zeros(W[k].shape))
        v_w.append(np.zeros(W[k].shape))
        m_b.append(np.zeros(b[k].shape))
        v_b.append(np.zeros(b[k].shape))
        t = i*n + j+1
        m_w[k] = (beta1*m_prev_w[k] + (1-beta1)*grad_W[k])/(1-np.power(beta1,t))
        v_w[k] = (beta2*v_prev_w[k] + (1-beta2)*np.square(grad_W[k]))/(1-np.power(beta2,t))
        m_b[k] = (beta1*m_prev_b[k] + (1-beta1)*grad_b[k])/(1-np.power(beta1,t))
        v_b[k] = (beta2*v_prev_b[k] + (1-beta2)*np.square(grad_b[k]))/(1-np.power(beta2,t))
        W_new.append((W[k] - (learn_rate*np.divide(m_w[k],(np.sqrt(v_w[k]+epsilon)))) - learn_rate*alpha*W[k]))
        b_new.append((b[k] - (learn_rate*np.divide(m_b[k],(np.sqrt(v_b[k]+epsilon))))))
        m_prev_w[k] = m_w[k]
        v_prev_w[k] = v_w[k]
        m_prev_b[k] = m_b[k]
        v_prev_b[k] = v_b[k]
      del m_w
      del v_w
      del m_b
      del v_b
      W = W_new
      b = b_new
    loss, acc = loss_and_accuracy(W, b, X_train, Y_train, act,alpha)
    val_loss, val_acc = loss_and_accuracy(W, b, X_validation, Y_validation, act,alpha)
    wandb.log({"loss": loss, "val_loss": val_loss, "accuracy": acc, "val_accuracy": val_acc, "epoch": i})
  return W, b

In [183]:
W, b = adam(10,0.01,0.9,0.99, 10**-8,32,'relu','xavier',N, 0.005)

2.3029489157353984 0.0999074074074074 2.302573494967874 0.10083333333333333
2.3029473640461413 0.0999074074074074 2.30256221236608 0.10083333333333333
2.30294596530191 0.0999074074074074 2.3025518675977144 0.10083333333333333
2.302944702870401 0.0999074074074074 2.302542380881369 0.10083333333333333
2.3029435621318832 0.0999074074074074 2.30253367951331 0.10083333333333333
2.3029425302100175 0.0999074074074074 2.302525697191158 0.10083333333333333
2.302941594955519 0.0999074074074074 2.302518371905815 0.10083333333333333
2.302938057970235 0.0999074074074074 2.3025076467687597 0.10083333333333333
1.3405430050371032 0.4672962962962963 1.3645938904648924 0.46216666666666667
0.5965437095951658 0.7805555555555556 0.6086987050297276 0.7848333333333334


In [156]:
def nadam(epochs, learn_rate, beta1, beta2, epsilon, batch_size, act, init, N, alpha):
  W, b = initialize_weights(N,init, act)
  m_prev_w = []
  m_prev_w.append(0)
  m_prev_b = []
  m_prev_b.append(0)
  v_prev_w = []
  v_prev_w.append(0)
  v_prev_b = []
  v_prev_b.append(0)
  n=(np.ceil(X_train.shape[1]/batch_size)).astype(int)
  for i in range(1,l+1):
    m_prev_w.append(np.zeros(W[i].shape))
    m_prev_b.append(np.zeros(b[i].shape))
    v_prev_w.append(np.zeros(W[i].shape))
    v_prev_b.append(np.zeros(b[i].shape))
  for i in range(epochs):
    for j in range(n):
      X=X_train[:,(j*batch_size):((j+1)*batch_size)]
      Y=Y_train[:,(j*batch_size):((j+1)*batch_size)]
      m_w = []
      m_w.append(0)
      v_w = []
      v_w.append(0)
      m_b = []
      m_b.append(0)
      v_b = []
      v_b.append(0)
      a, h, y_hat = feedforward(X, W, b, N, l, act)
      W_new =[]
      W_new.append(0)
      b_new =[]
      b_new.append(0)
      for k in range(1,l+1):
        m_w.append(np.zeros(W[k].shape))
        v_w.append(np.zeros(W[k].shape))
        m_b.append(np.zeros(b[k].shape))
        v_b.append(np.zeros(b[k].shape))
        t = i*n + j+1
        m_w[k] = (beta1*m_prev_w[k])/(1-np.power(beta1,t))
        v_w[k] = (beta2*v_prev_w[k])/(1-np.power(beta2,t))
        m_b[k] = (beta1*m_prev_b[k])/(1-np.power(beta1,t))
        v_b[k] = (beta2*v_prev_b[k])/(1-np.power(beta2,t))
        W_new.append((W[k] - (learn_rate*np.divide(m_w[k],(np.sqrt(v_w[k]+epsilon))))))
        b_new.append((b[k] - (learn_rate*np.divide(m_b[k],(np.sqrt(v_b[k]+epsilon))))))
      W = W_new
      b = b_new
      grad_W, grad_b = backprop(X, Y, y_hat, W, a, h, act, alpha)
      for k in range(1,l+1):
        m_w[k] = (beta1*m_prev_w[k] + (1-beta1)*grad_W[k])/(1-np.power(beta1,t))
        v_w[k] = (beta2*v_prev_w[k] + (1-beta2)*np.square(grad_W[k]))/(1-np.power(beta2,t))
        m_b[k] = (beta1*m_prev_b[k] + (1-beta1)*grad_b[k])/(1-np.power(beta1,t))
        v_b[k] = (beta2*v_prev_b[k] + (1-beta2)*np.square(grad_b[k]))/(1-np.power(beta2,t))
        W_new[k] = W[k] - (learn_rate*np.divide(m_w[k],(np.sqrt(v_w[k]+epsilon)))) - learn_rate*alpha*W[k]
        b_new[k] = b[k] - (learn_rate*np.divide(m_b[k],(np.sqrt(v_b[k]+epsilon))))
        m_prev_w[k] = m_w[k]
        v_prev_w[k] = v_w[k]
        m_prev_b[k] = m_b[k]
        v_prev_b[k] = v_b[k]
      del m_w
      del v_w
      del m_b
      del v_b
      W = W_new
      b = b_new
    loss, acc = loss_and_accuracy(W, b, X_train, Y_train, act,alpha)
    val_loss, val_acc = loss_and_accuracy(W, b, X_validation, Y_validation, act,alpha)
    wandb.log({"loss": loss, "val_loss": val_loss, "accuracy": acc, "val_accuracy": val_acc, "epoch": i})
  return W, b

In [ ]:
W, b = nadam(10, 0.1,0.9,0.99,10**-8,32,'sigmoid','random',N, 0.005, 1)

In [146]:
sweep_config = {
    'method': 'grid'
    }

In [207]:
parameters_dict = {
    # 'l': {
    #     'values': [3, 4, 5]
    #   },
    # 'size of each layer': {
    #     'values': [32, 64, 128]
    #   },
    'alpha': {
        'values': [0.0005]
      },
    'learn_rate': {
        'values': [1e-1]
      },
    'epochs': {
        'values': [10]
      },
    'batch_size': {
        'values': [32]
      },
    'init': {
        'values': ['random', 'xavier']
      },
    'optimizer': {
        'values': ['sgd', 'mom', 'nest', 'rms', 'adam', 'nadam']
      },
    'act': {
          'values': ['sigmoid','tanh','relu']
      }
    }

sweep_config['parameters'] = parameters_dict

In [208]:
import pprint

pprint.pprint(sweep_config)

{'method': 'grid',
 'parameters': {'act': {'values': ['sigmoid', 'tanh', 'relu']},
                'alpha': {'values': [0.0005]},
                'batch_size': {'values': [32]},
                'epochs': {'values': [10]},
                'init': {'values': ['random', 'xavier']},
                'learn_rate': {'values': [0.1]},
                'optimizer': {'values': ['sgd',
                                         'mom',
                                         'nest',
                                         'rms',
                                         'adam',
                                         'nadam']}}}


In [204]:
sweep_id = wandb.sweep(sweep_config, project="CS6910-assg1")

Create sweep with ID: 64zr6st6
Sweep URL: https://wandb.ai/swathi/CS6910-assg1/sweeps/64zr6st6


In [205]:
wandb.agent(sweep_id, training_sweep)

wandb: Agent Starting Run: 82o2zk7r with config:
wandb: 	act: sigmoid
wandb: 	alpha: 0.0005
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	init: random
wandb: 	learn_rate: 0.01
wandb: 	optimizer: sgd


loss,0.88126
val_loss,0.93006
accuracy,0.70298
val_accuracy,0.70633
epoch,9
_runtime,57
_timestamp,1615631438
_step,9


loss,█▆▅▄▃▂▂▁▁▁
val_loss,█▆▅▄▃▂▂▁▁▁
accuracy,▁▄▆▆▇▇▇███
val_accuracy,▁▄▆▆▇▇▇███
epoch,▁▂▃▃▄▅▆▆▇█
_runtime,▁▂▂▃▄▅▆▆▇█
_timestamp,▁▂▂▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: 5tefylyl with config:
wandb: 	act: sigmoid
wandb: 	alpha: 0.0005
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	init: random
wandb: 	learn_rate: 0.01
wandb: 	optimizer: mom


loss,1.40272
val_loss,1.44819
accuracy,0.58087
val_accuracy,0.58433
epoch,9
_runtime,66
_timestamp,1615631510
_step,9


...

loss,███████▄▂▁
val_loss,███████▄▂▁
accuracy,▁▁▁▁▁▁▁▄▇█
val_accuracy,▁▁▁▁▁▁▁▅▇█
epoch,▁▂▃▃▄▅▆▆▇█
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Agent Starting Run: 3b6chi5g with config:
wandb: 	act: relu
wandb: 	alpha: 0.0005
wandb: 	batch_size: 32
wandb: 	epochs: 10
wandb: 	init: xavier
wandb: 	learn_rate: 0.01
wandb: 	optimizer: nadam


loss,0.95301
val_loss,0.96218
accuracy,0.60263
val_accuracy,0.603
epoch,9
_runtime,98
_timestamp,1615634319
_step,9


loss,███████▆▁▁
val_loss,███████▆▁▁
accuracy,▁▁▁▁▁▁▁▂██
val_accuracy,▁▁▁▁▁▁▁▂██
epoch,▁▂▃▃▄▅▆▆▇█
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [203]:
def training_sweep(config=None):
  
  # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        if config.optimizer == 'sgd':
          gradient_descent(config.epochs, config.learn_rate, config.batch_size, config.act, config.init, N, config.alpha)
        if config.optimizer == 'mom':
          mom_grad_descent(config.epochs, config.learn_rate, 0.9, config.batch_size, config.act, config.init, N, config.alpha)
        if config.optimizer == 'nest':
          nesterov_grad_descent(config.epochs, config.learn_rate, 0.9, config.batch_size, config.act, config.init, N, config.alpha)
        if config.optimizer == 'rms':
          rms_prop(config.epochs, config.learn_rate, 0.9, 10**-8,config.batch_size, config.act, config.init, N, config.alpha)
        if config.optimizer == 'adam':
          adam(config.epochs, config.learn_rate, 0.9, 0.99, 10**-8,config.batch_size, config.act, config.init, N, config.alpha)
        if config.optimizer == 'nadam':
          nadam(config.epochs, config.learn_rate, 0.9, 0.99, 10**-8,config.batch_size, config.act, config.init, N, config.alpha)

        #wandb.log({"loss": avg_loss, "epoch": epoch})